In [2]:
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
import tensorflow as tf

import tensorflow.keras.backend as K
from keras.models import Sequential,Model
from keras.layers import Conv2D,MaxPooling2D
from keras.layers import Activation, Dense, Flatten, Dropout
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.utils import plot_model

from os import listdir
from os.path import isfile, join
from PIL import Image
import glob

%matplotlib inline

Using TensorFlow backend.


In [0]:
PARAMS = {'img_dims': 160,
          'img_dims_inception': 299,
          'img_dims_vgg': 224,
          'epochs': 100,
          'batch_size': 64,
          }

## Loading of Data

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [5]:
!ls '/content/drive/My Drive/Colab Notebooks/Data/OCT 2017'

test  train  val


In [0]:
train_path = '/content/drive/My Drive/Colab Notebooks/Data/OCT 2017/train'

test_path = '/content/drive/My Drive/Colab Notebooks/Data/OCT 2017/test'

val_path = '/content/drive/My Drive/Colab Notebooks/Data/OCT 2017/val'

In [0]:
def process_test_data(val_path, img_dim):
  test_datagen = ImageDataGenerator(rescale=1./255, horizontal_flip=False)
  test_gen = test_datagen.flow_from_directory(directory=val_path, 
                                                target_size=(img_dim,img_dim),
                                                class_mode='categorical',
                                                shuffle=False)
  return test_gen

In [0]:
def process_data(img_dim, batch_size):
    # Split 0.2
    train_datagen = ImageDataGenerator(rescale=1./255,
                                       zoom_range=0.3, horizontal_flip=True, validation_split=0.2)
    
    val_datagen = ImageDataGenerator(rescale=1./255, validation_split=0.2)
    
    train_gen = train_datagen.flow_from_directory(directory=test_path, 
                                                  target_size=(img_dim,img_dim), 
                                                  batch_size=batch_size, 
                                                  class_mode = 'categorical', 
                                                  subset='training',
                                                  shuffle=True)

    val_gen = val_datagen.flow_from_directory(directory=test_path, 
                                                target_size=(img_dim,img_dim), 
                                                batch_size=batch_size, 
                                                class_mode='categorical', 
                                                subset='validation',
                                                shuffle=True)
    
    return train_gen, val_gen

In [9]:
from keras.applications.vgg16 import VGG16
from keras.applications.mobilenet import MobileNet
from keras.applications.resnet_v2 import ResNet50V2
from keras.applications.inception_v3 import InceptionV3
from keras.applications.inception_resnet_v2 import InceptionResNetV2

# model = InceptionV3(weights = "imagenet", include_top=False, input_shape=(PARAMS['img_dims_inception'], PARAMS['img_dims_inception'], 3))
# model = InceptionResNetV2(weights = "imagenet", include_top=False, input_shape=(PARAMS['img_dims_inception'], PARAMS['img_dims_inception'], 3))
model = VGG16(weights = "imagenet", include_top=False, input_shape=(PARAMS['img_dims_vgg'], PARAMS['img_dims_vgg'], 3))
# model = MobileNet(weights = "imagenet", include_top=False, input_shape=(PARAMS['img_dims_vgg'], PARAMS['img_dims_vgg'], 3))
# model = ResNet50V2(weights = "imagenet", include_top=False, input_shape=(PARAMS['img_dims_vgg'], PARAMS['img_dims_vgg'], 3))

# Freeze front layers
for layer in model.layers:
    layer.trainable = False

# Adding FC layers 
x = model.output
x = Flatten()(x)
x = Dropout(0.5)(x)
x = Dense(256, activation="relu")(x)
predictions = Dense(4, activation="softmax")(x)

model = Model(inputs=model.input, outputs=predictions)

58892288/58889256 [==============================] - 2s 0us/step


In [0]:
opt = Adam(lr=0.0002)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [0]:
# if inception use:
# train_gen, val_gen = process_data(PARAMS['img_dims_inception'], PARAMS['img_dims_inception'])
# if VGG, MobileNet or ResNet, use:
train_gen, val_gen = process_data(PARAMS['img_dims_vgg'], PARAMS['img_dims_vgg'])


In [0]:
# use the appropriate image size
# test_gen = process_test_data(val_path, PARAMS['img_dims_inception'])
test_gen = process_test_data(val_path, PARAMS['img_dims_vgg'])

In [0]:
import keras

es = keras.callbacks.EarlyStopping(monitor='val_loss',mode='min',patience=5)

In [14]:
history = model.fit_generator(train_gen, epochs=100,
                              steps_per_epoch=train_gen.samples // PARAMS['batch_size'],
                              validation_data=val_gen,
                              validation_steps=30,
                              callbacks=[es])

Epoch 1/100


/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 3 could not be retrieved. It could be because a worker has died.
  UserWarning)


 1/12 [=>............................] - ETA: 11:09 - loss: 2.2049 - accuracy: 0.2523

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 1 could not be retrieved. It could be because a worker has died.
  UserWarning)


 2/12 [====>.........................] - ETA: 14:59 - loss: 2.0921 - accuracy: 0.2746

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 2 could not be retrieved. It could be because a worker has died.
  UserWarning)


 3/12 [======>.......................] - ETA: 15:06 - loss: 2.1070 - accuracy: 0.2630

/usr/local/lib/python3.6/dist-packages/keras/utils/data_utils.py:616: UserWarning: The input 0 could not be retrieved. It could be because a worker has died.
  UserWarning)


12/12 [==============================] - 468s 39s/step - loss: 1.5096 - accuracy: 0.3823 - val_loss: 1.0317 - val_accuracy: 0.5464
Epoch 2/100
12/12 [==============================] - 49s 4s/step - loss: 0.8697 - accuracy: 0.6594 - val_loss: 0.6178 - val_accuracy: 0.7990
Epoch 3/100
12/12 [==============================] - 48s 4s/step - loss: 0.5612 - accuracy: 0.8084 - val_loss: 0.4852 - val_accuracy: 0.8557
Epoch 4/100
12/12 [==============================] - 48s 4s/step - loss: 0.4141 - accuracy: 0.8514 - val_loss: 0.3695 - val_accuracy: 0.8557
Epoch 5/100
12/12 [==============================] - 48s 4s/step - loss: 0.3241 - accuracy: 0.8966 - val_loss: 0.3061 - val_accuracy: 0.8918
Epoch 6/100
12/12 [==============================] - 48s 4s/step - loss: 0.2682 - accuracy: 0.9080 - val_loss: 0.2590 - val_accuracy: 0.9175
Epoch 7/100
12/12 [==============================] - 48s 4s/step - loss: 0.2430 - accuracy: 0.9217 - val_loss: 0.2176 - val_accuracy: 0.9227
Epoch 8/100
12/12 [====

In [15]:
model.evaluate(test_gen)

1/1 [==============================] - 16s 16s/step


[0.15945149958133698, 0.9375]